# Python-DB 연결 실습

## Pandas 튜토리얼

- [pandas Getting started tutorials](https://pandas.pydata.org/docs/getting_started/intro_tutorials/index.html)

In [ ]:
# pandas 라이브러리 import
import pandas as pd

### Pandas read_csv()
Lecture_14_data.zip 에 있는 orders.csv 파일을 이용하여 pandas로 csv 파일을 읽습니다.

Google Colab을 사용하시는 분들은 왼쪽 메뉴 중 파일 메뉴에 orders.csv 파일을 업로드하시면 되고,

Jupyter Notebook을 사용하시는 분들은 orders.csv 파일을 실습 파일과 같은 폴더에 넣으시면 됩니다.

- [read_csv documentation](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)


In [ ]:
df_csv = pd.read_csv('orders.csv')

In [ ]:
#df_csv 파일 확인
df_csv

## BigQuery 연결 실습

pandas-gbq를 사용하여 Google BigQuery와 연결합니다.
- [Documentation](https://pandas-gbq.readthedocs.io/en/latest/)

In [ ]:
# Local Jupyter Notebook을 사용하시는 분들은 아래 코드를 uncomment하여 pandas-gbq를 설치하셔야합니다.

# !pip install pandas-gbq

In [ ]:
# gbq import
from pandas.io import gbq

아래 코드를 수정 후 실행하면, 계정 인증 후 authorization code를 복사하여 붙여넣을 수 있습니다.

In [ ]:
import pydata_google_auth

SCOPES = [
    'https://www.googleapis.com/auth/cloud-platform',
    'https://www.googleapis.com/auth/bigquery'
]

credentials = pydata_google_auth.get_user_credentials(
  SCOPES, auth_local_webserver=False)

# SQL을 사용하여 프로젝트에서 데이터를 조회 후 pandas dataframe에 입력

sql = "SELECT * FROM census.input_view" # Lecture 18에서 생성한 input_view 활용
project_id = "본인의 프로젝트 ID"

df = pd.read_gbq(sql, project_id=project_id, credentials=credentials, dialect='standard')

Import된 dataframe 확인

In [ ]:
df

참고) pandas dataframe을 BigQuery에 업로드할 수 있습니다. (**billing 설정시에만 가능**)

In [ ]:
df_csv.to_gbq(destination_table='데이터셋명.테이블명',
		  project_id='프로젝트 ID',
		  if_exists='replace')

## PostgreSQL Server 연결 실습
psycopg2를 사용하여 PostgreSQL server와 연결합니다.

우선, psycopg2를 설치해야합니다.

In [ ]:
!pip install psycopg2-binary

설치가 완료되면, 아래 코드를 사용해 연결 설정을 해주세요.

In [ ]:
# psycopg2 import
import psycopg2

# 연결정보 입력
connection_info = "host=147.47.200.145 dbname=<팀데이터베이스명> user=<팀유저명> password=<팀패스워드> port=34543"

DBeaver에 exercise2_data.txt의 쿼리를 사용하여 PostgreSQL 팀 데이터베이스에 테이블을 생성하고, (**팀당 한 명만**)

아래 코드를 사용해 PostgreSQL에 연결해 데이터를 읽어옵니다.

In [ ]:
# PostgreSQL 연결
conn = psycopg2.connect(connection_info)

try:
    # 테이블을 Pandas.Dataframe으로 추출
    df = pd.read_sql('SELECT * FROM researchers ORDER BY rid ',conn)

except psycopg2.Error as e:
    # 데이터베이스 에러 처리
    print("DB error: ", e)
    
finally:
    # 데이터베이스 연결 해제 필수!!
    conn.close()

In [ ]:
# 데이터 확인
df

### 트랜잭션 쿼리 수행

만약 테이블이나 값을 변경하는 쿼리를 수행했을경우, commit을 해줘야 변경사항이 데이터베이스에 반영됩니다.


In [ ]:
# PostgreSQL 연결
conn = psycopg2.connect(connection_info)

try:
    # 커서(Cursor) 생성
    # 커서: 명령문(SQL) 실행, 결과의 현재 위치 표시
    cursor = conn.cursor()
    
    # 값 변경 쿼리
    cursor.execute("UPDATE public.researchers set lab = 'BKMS' WHERE rid = 1;")
    
    # 트랜잭션 커밋 - 데이터베이스에 업데이트를 반영
    conn.commit()

except psycopg2.Error as e:
    # 데이터베이스 에러 처리
    print("DB error: ", e)
    # 롤백- 최근 커밋 이후의 transaction들을 모두 취소
    conn.rollback()
    
finally:
    # 데이터베이스 연결 해제 필수!!
    conn.close()

In [ ]:
# PostgreSQL 연결
conn = psycopg2.connect(connection_info)

try:
    # 테이블을 Pandas.Dataframe으로 추출
    df = pd.read_sql('SELECT * FROM researchers ORDER BY rid ',conn)
    print(df.head())

except psycopg2.Error as e:
    # 데이터베이스 에러 처리
    print("DB error: ", e)
    
finally:
    # 데이터베이스 연결 해제 필수!!
    conn.close()

아래 코드를 실행하여 다시 원래 값으로 업데이트합니다.

In [ ]:
# PostgreSQL 연결
conn = psycopg2.connect(connection_info)

try:
    # 커서(Cursor) 생성
    # 커서: 명령문(SQL) 실행, 결과의 현재 위치 표시
    cursor = conn.cursor()
    
    # 값 변경 쿼리
    cursor.execute("UPDATE public.researchers set lab = 'YUHL' WHERE rid = 1;")
    
    # 트랜잭션 커밋 - 데이터베이스에 업데이트를 반영
    conn.commit()

except psycopg2.Error as e:
    # 데이터베이스 에러 처리
    print("DB error: ", e)
    # 롤백- 최근 커밋 이후의 transaction들을 모두 취소
    conn.rollback()
    
finally:
    # 데이터베이스 연결 해제 필수!!
    conn.close()

## Neo4j Server 연결 실습

Neo4j driver가 지원하는 기능이 많지 않아서 사실 Neo4j Browser에서 Cypher 쿼리를 돌리는 것이 더 낫습니다.

하지만 데이터 Flow를 자동화하는 상황에서 아래 코드들을 사용할 수 있겠습니다.

In [ ]:
# neo4j 패키지 설치
!pip install neo4j

In [ ]:
# neo4j 드라이버 import 및 함수 정의
from neo4j import GraphDatabase

class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

아래 코드에 Neo4j 연결 정보를 설정해주세요.

In [ ]:
dbname = "팀 DB이름"
uri_param = "bolt://147.47.200.145:37687"
user_param = "팀username"
pwd_param = "팀비밀번호"

아래 쿼리를 통해 연결을 테스트해봅니다.

In [ ]:
# Neo4j 연결
conn = Neo4jConnection(uri=uri_param, user=user_param, pwd=pwd_param)

# Cypher 쿼리 입력
cypher = 'MATCH (n) return n'

# Cypher 쿼리 실행 후 결과를 response에 저장
response = conn.query(cypher, db=dbname)

# 연결 종료 필수!
conn.close()

결과 확인 (현재 그래프가 비어있으므로 [ ] 가 나와야 합니다.)

In [ ]:
response

Neo4j와 PostgreSQL이 같은 서버에 설치되어 있으므로, APOC를 사용하여 데이터를 불러올 때 아래와 같이 localhost로 접속하면됩니다.

우선, 아래 코드에 PostgreSQL 연결 정보를 입력해주세요.

In [ ]:
apoc_string = 'CALL apoc.load.jdbc("jdbc:postgresql://localhost:34543/<DB이름>?user=<유저명>&password=<PostgreSQL 비밀번호>",'

아래 세 코드를 사용해 Academic Database를 Neo4j에 import합니다.

In [ ]:
# Neo4j 연결
conn = Neo4jConnection(uri=uri_param, user=user_param, pwd=pwd_param)

# Cypher 쿼리 입력
cypher = apoc_string + '"researchers") YIELD row MERGE (r:Researcher {rid: toInteger(row.rid), rname:row.rname}) MERGE (l:Lab {name:row.lab}) MERGE (r)-[rel:BELONGSTO]->(l) RETURN count(r)'

# Cypher 쿼리 실행 후 결과를 print
print(conn.query(cypher, db=dbname))

# 연결 종료 필수!
conn.close()

In [ ]:
# Neo4j 연결
conn = Neo4jConnection(uri=uri_param, user=user_param, pwd=pwd_param)

# Cypher 쿼리 입력
cypher = apoc_string + '"papers") YIELD row MERGE (p:Paper {pid: toInteger(row.pid), title:row.title, pyear:toInteger(row.pyear), citations:toInteger(row.citations)}) return count(p)'

# Cypher 쿼리 실행 후 결과를 print
print(conn.query(cypher, db=dbname))

# 연결 종료 필수!
conn.close()

In [ ]:
# Neo4j 연결
conn = Neo4jConnection(uri=uri_param, user=user_param, pwd=pwd_param)

# Cypher 쿼리 입력
cypher = apoc_string + '"wrote") YIELD row MATCH (r:Researcher {rid: toInteger(row.rid)}) MATCH (p:Paper {pid: toInteger(row.pid)}) MERGE (r)-[w:WROTE {author_seq:toInteger(row.author_seq)}]->(p) return count(w)'

# Cypher 쿼리 실행 후 결과를 print
print(conn.query(cypher, db=dbname))

# 연결 종료 필수!
conn.close()

아래 코드를 실행하여 쿼리 결과를 Python으로 확인합니다.

In [ ]:
# Neo4j 연결
conn = Neo4jConnection(uri=uri_param, user=user_param, pwd=pwd_param)

# Cypher 쿼리 입력
cypher = 'MATCH (n) RETURN n LIMIT 25'

# Cypher 쿼리 실행 후 결과를 print
print(conn.query(cypher, db=dbname))

# 연결 종료 필수!
conn.close()